In [1]:
import numpy as np
from lookup_table import CaseNum2EdgeOffset, getCaseNum
import trimesh
import os
import time

In [2]:
def marching_cube(thres,cells):
    # vertices use dictionary to avoid duplicate axes
    vertex_array = {}
    face_array = []
    t1 = time.time()
    # -------------------TODO------------------ 
    # compute vertices and faces
    # vertices: [N, 3]
    # faces: [M, 3], e.g. np.array([[0,1,2]]) means a triangle composed of vertices[0], vertices[1] and vertices[2]
    # for-loop is allowed to reduce difficulty
    # -------------------TODO------------------ 
    def interpolation(x1,y1,z1,x2,y2,z2):
        v1=cells[x1][y1][z1]
        v2=cells[x2][y2][z2]
        u=v1/(v1-v2)
        xp=x1+u*(x2-x1)
        yp=y1+u*(y2-y1)
        zp=z1+u*(z2-z1)
        return np.array([xp,yp,zp])
    
    x_len,y_len,z_len=cells.shape
    vertex_cnt=0 # 点的索引数
    face_three_cnt=0 # 记录是否找到一个面的三个点
    face_vertex_lst=[0,0,0] # 滚动数组存储面的三个点的索引
    axis2vertex_idx={} # 从边到点索引的字典

    for x in range(x_len-1):
        for y in range(y_len-1):
            for z in range(z_len-1):
                case_nums=getCaseNum(x,y,z,thres,cells)
                for case_num in case_nums:
                    if case_num!=-1:
                        x1=x+CaseNum2EdgeOffset[case_num][0]
                        y1=y+CaseNum2EdgeOffset[case_num][1]
                        z1=z+CaseNum2EdgeOffset[case_num][2]
                        x2=x+CaseNum2EdgeOffset[case_num][3]
                        y2=y+CaseNum2EdgeOffset[case_num][4]
                        z2=z+CaseNum2EdgeOffset[case_num][5]
                        axis=(x1,y1,z1,x2,y2,z2)
                        if axis not in axis2vertex_idx:
                            interaction_point=interpolation(x1,y1,z1,x2,y2,z2)
                            vertex_array[vertex_cnt]=interaction_point
                            axis2vertex_idx[axis]=vertex_cnt
                            vertex_cnt+=1
                        face_vertex_lst[face_three_cnt]=axis2vertex_idx[axis]
                        face_three_cnt+=1
                        if face_three_cnt==3:
                            face_array.append([face_vertex_lst[0],face_vertex_lst[1],face_vertex_lst[2]])
                            face_three_cnt=0
    t2 = time.time()
    print("\nTime taken by algorithm\n"+'-'*40+"\n{} s".format(t2-t1))
    vertex_array = list(vertex_array.values())
    return np.array(vertex_array), np.array(face_array)

In [3]:
# reconstruct these two animals
shape_name_lst = ['spot', 'bob']
for shape_name in shape_name_lst:
    data = np.load(os.path.join('data', shape_name + '_cell.npy'))
    verts, faces = marching_cube(0, data)
    mesh = trimesh.Trimesh(vertices=verts, faces=faces)
    mesh_txt = trimesh.exchange.obj.export_obj(mesh)
    with open(os.path.join('../results', shape_name + '.obj'),"w") as fp:
        fp.write(mesh_txt)


Time taken by algorithm
----------------------------------------
4.312490463256836 s

Time taken by algorithm
----------------------------------------
4.345934629440308 s
